# Convolutional Network that classifies two images as similar or disimilar

# Imports

In [ ]:
# # # Credit: https://stackoverflow.com/questions/34199233/how-to-prevent-tensorflow-from-allocating-the-totality-of-a-gpu-memory
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
from tensorflow import keras
from loaders import TripletSequence
from loaders import PairSequence
import nets
import data

# Globals

In [ ]:
K = keras.backend
codings_size = 10
alpha = 0.5

# Data

In [ ]:
emnist = data.load_dataset('emnist/balanced')
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = emnist.split()

In [ ]:
train_seq = PairSequence(train_x, train_y, samples_per_label=1, batch_size=1024, can_shuffle=True, output='y')
valid_seq = PairSequence(valid_x, valid_y, samples_per_label=1, batch_size=1024, can_shuffle=True, output='y')
test_seq  = PairSequence(test_x,  test_y,  samples_per_label=1, batch_size=1024, can_shuffle=True, output='y')

# Model

In [ ]:
conv_classifier = nets.make_conv_classifier(codings_size)

In [ ]:
opt = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9, nesterov=True)

In [ ]:
conv_classifier.compile(loss='binary_crossentropy', 
                    metrics=['accuracy'],
                    optimizer=opt 
                   )   


In [ ]:
early_stopping = keras.callbacks.EarlyStopping(patience=20, min_delta=1/100000, restore_best_weights=True, monitor='val_loss')

In [ ]:
conv_classifier.fit(train_seq, epochs=500, validation_data=valid_seq, workers=26, callbacks=[early_stopping])

# Evaluation

In [ ]:
for i in range(3):
    conv_classifier.evaluate(train_seq)

In [ ]:
for i in range(3):
    conv_classifier.evaluate(valid_seq)

In [ ]:
for i in range(3):
    conv_classifier.evaluate(test_seq)

In [ ]:
print("Saving Model")
conv_classifier.save("conv_model")